In [1]:
import requests
import urllib.request
import json

from SPARQLWrapper import SPARQLWrapper, JSON


In [2]:
# Fonctions utiles

# Pour enlever la qualificatif entre () dans un nom provenant de DBpedia
def traiter_nom(chaine):
    c = chaine.split(' (')
    return c[0]

def ajouter_annee_naissance(nom,annee):
    return nom + ' ' + annee


def trouver_ecrivain(ecrivain,liste_ecrivains):
    for e in liste_ecrivains:
        if 'dbpedia-fr' in ecrivain and 'dbpedia-fr' in e:
            if ecrivain['dbpedia-fr'] == e['dbpedia-fr']:
                return e
        if 'dbpedia-en' in ecrivain and 'dbpedia-en' in e:
            if ecrivain['dbpedia-en'] == e['dbpedia-en']:
                return e
        
    return None

# Sites intéressants à explorer

* Wikisource https://fr.wikisource.org/wiki/Wikisource
* L'ÎLE http://www.litterature.org/
* http://www.litterature-quebecoise.com/ Site maintenu par un professeur à la retraite
* Centre des auteurs dramatiques http://www.cead.qc.ca/_accueil
* Auteurs contemporains https://auteurs.contemporain.info/doku.php/accueil
* L'encyclopédie canadienne https://thecanadianencyclopedia.ca/fr
* Regroupement des éditeurs franco-canadiens (ressource pour les enseignants) https://www.copibecnumerique.ca/home
* Le catalogue du Système Universitaire de Documentation http://www.sudoc.abes.fr/
* Sites des maisons d'édition
* Site des libraires
    * https://www.leslibraires.ca/
* Association nationale des éditeurs de livres https://www.anel.qc.ca/
* Sites pour lecteurs
    * Babelio https://www.babelio.com/
    * Pause lecture http://www.pauselecture.net/
* Érudit https://www.erudit.org/fr/
* Papyrus - Dépôt institutionnel de l'UdeM https://papyrus.bib.umontreal.ca/
* Revues et journaux
    * Canadian litterature https://canlit.ca/
    * Revue Les libraires https://revue.leslibraires.ca/
    * Nuit blanche https://nuitblanche.com/
    * Moebius https://revuemoebius.com/
    * Lettres québécoises http://lettresquebecoises.qc.ca/
    * Revue Zinc https://revuezinc.com/
    
* Sites des bibliothèques
* Les poètes de l'Amérique française (spectacles littéraires) http://lespaf.org/
* UNEQ https://www.uneq.qc.ca/

Poésie:
* http://www.sculfort.fr/articles/litterature/poemes/poesiequebecoise.html
* https://www.lesvoixdelapoesie.com
* http://www.toutacouplapoesie.ca/liens
* http://faculty.marianopolis.edu/c.belanger/quebechistory/encyclopedia/roylitchap10.htm
* https://www.lecourriersud.com/50-ans-du-poeme-speak-white-des-lectures-publiques-aux-quatre-coins-de-la-region/
* https://www.wikipoemes.com/

In [3]:
# Initialisation la liste de tous les écrivain
# Pour chaque écrivain on voudra son URI
#      dans Wikidata
#      dans DBpedia français
#      dans DBpedia anglais
#      dans UNEQ
#      dans ILE

# Liste des écrivains, chaque item sera une liste, parce qu'il se peut que deux écrivains aient le
# même nom, et on prend le nom comme clé
ecrivains = {}

# Ressources web intéressantes

## DBpedia

### Recherche des auteurs dans DBpedia français

Une première approche pour trouver les auteurs dans DBpedia consiste à trouver ceux qui sont associés à une catégorie désignant un auteur:

* Écrivain québécois
* Poète québécois
* Dramaturge québécois
* ...

On note que les catégories forment une hiérarchie dominée par la catégorie "Écrivain québécois" 

On note aussi que plusieurs auteurs ont un URI, mais ne sont le sujet d'aucun triplet dans DBpedia. On peut les retrouver en recherchant ceux qui sont l'objet de la relation <http://dbpedia.org/ontology/wikiPageWikiLink> 
avec un prix littéraire comme sujet. Le problème ici est que cette relation est vague et que l'objet peut 
être bien autre chose qu'une personne. 







In [4]:
# Recherche dans DBpedia français
sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setQuery("""
  PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  SELECT DISTINCT ?s ?label ?naissance
  WHERE {
    ?s dcterms:subject ?cat .
    ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_québécois> .
  
    ?s  rdfs:label ?label . 
    OPTIONAL { ?s <http://dbpedia.org/ontology/birthDate> ?naissance .}
    FILTER (lang(?label) = "fr")
  } 
""")

sparql.setReturnFormat(JSON)
results_dbpedia_fr = sparql.query().convert()

In [5]:
filtre = ["Liste d'écrivains québécois par ordre alphabétique",
           "Association des écrivaines et des écrivains québécois pour la jeunesse",
           "Liste de poètes québécois",
           "Liste d'auteurs québécois par ordre chronologique",
           "Liste d'auteurs québécois par ordre chronologique de décès"]


# On récupère les données extraites par la requête SPARQL
ecrivains_dbpedia_fr = set([(b['s']['value'],
                         traiter_nom(b['label']['value']), 
                         b['naissance']['value'][:4] if 'naissance' in b else 'X') 
                        for b in results_dbpedia_fr['results']['bindings']])


# On retire ceux qui ne correspondent pas à des écrivains
ecrivains_dbpedia_fr = [(uri,nom,naiss) for (uri,nom,naiss) in ecrivains_dbpedia_fr if nom not in filtre]

print(len(ecrivains_dbpedia_fr), 'écrivains du Québec trouvés dans DBpedia français')


# On ajoute à la liste des écrivains 

ecrivains_temp = {}
for e in ecrivains_dbpedia_fr:
    nom = e[1]
 
    ecrivain = {}
    ecrivain['dbpedia-fr'] = e[0]
    ecrivain['naissance'] = e[2] 

    if nom not in ecrivains_temp:
        ecrivains_temp[nom] = [ecrivain]
    else:
        if not trouver_ecrivain(ecrivain, ecrivains_temp[nom]):
            ecrivains_temp[nom].append(ecrivain)
    
for e in ecrivains_temp:
    if e not in ecrivains:
        ecrivains[e] = ecrivains_temp[e]


# for e in list(ecrivains.items()):
#     print(e)
        

file_writers = open("./ecrivains_dbpedia_fr.txt","w")
for e in ecrivains_dbpedia_fr:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()

print(len(ecrivains),'noms d\'écrivains distincts')
print(len([e for es in ecrivains.values() for e in es]),'écrivains au total')
print(len([e for es in ecrivains.values() for e in es if e['naissance'] != 'X']), 'ecrivains ont une date de naissence spécifiée')

print("voici la liste des écrivains qui ont le même nom:")
for e in ecrivains:
    if len(ecrivains[e]) > 1:
        print('   ', e, '({})'.format(len(ecrivains[e])))

1259 écrivains du Québec trouvés dans DBpedia français
1253 noms d'écrivains distincts
1259 écrivains au total
281 ecrivains ont une date de naissence spécifiée
voici la liste des écrivains qui ont le même nom:
    Pierre Morency (2)
    Jean-François Caron (2)
    Michel David (2)
    Louis Émond (2)
    Bruno Roy (2)
    Gilbert Dupuis (2)


On note la présence de 6 cas d'homonymie, dont 4 qu'on ne peut pas les distinguer par la date de naissance, qui est inconnue pour les deux:

* Bruno Roy  (médiéviste, 1935-)  (écrivain, professeur, poète, 1943-2010)
* Louis Émond (écrivain jeunesse, 1957-) (écrivain, 1969-)
* Pierre Morency (poète romancier, 1942) (essayiste, page supprimée dans Wikipédia)
* Gilbert Dupuis (dramaturge, 1953-) (écrivain, 1947)

Les deux cas qu'on peut distinguer sont les suivants:

* Jean-François Caron
* Michel David



### Recherche des auteurs dans DBpedia anglais

Plusieurs auteurs se retrouvent de DBpedia anglais. Même, certains sont présents seulement dans DBpedia anglais.
Pour les retrouver, on pourrait chercher ceux qui sont liés à la catégorie Writers_from_Quebec, ou une de ses sous-catégories. Mais cela couvrerait trop large, puisqu'il y a des sous-catégories, comme "McGill University alumni", qui ne représentent pas nécessairement des écrivains. Il faut donc lister toutes les sous-catégories pertinentes. Si on essaie d'utiliser toute la hiérarchie de Writers_from_Quebec et filtrer par la suite les instances du sous-arrbre dominé par la catégorie Academics_in_Quebec, ça ne marche pas, à cause d'un problème de mémoire sur le serveur.

À noter que l'on retrouve plus systématiquement les dates de naissance dans DBpedia anglais.
    
    

In [18]:

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   SELECT DISTINCT ?s ?label ?naissance  
   WHERE {
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Quebec> }
     UNION
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Montreal> }
     UNION
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Quebec_City>}

     ?s rdfs:label ?label . 
     OPTIONAL { ?s <http://dbpedia.org/ontology/birthDate> ?naissance .}
     
     FILTER (lang(?label) IN ("fr", "en"))
    } 
""")
sparql.setReturnFormat(JSON)
results_dbpedia_en = sparql.query().convert()



In [19]:
ecrivains_dbpedia_en = set([(b['s']['value'],
                             traiter_nom(b['label']['value']),
                             b['naissance']['value'][:4] if 'naissance' in b else 'X')
                            for b in results_dbpedia_en['results']['bindings']])
print(len(ecrivains_dbpedia_en), 'écrivains du Québec trouvés dans DBpedia anglais')



ecrivains_temp = {}

# On ajoute à la liste des écrivains 
for e in ecrivains_dbpedia_en:
    nom = e[1]
  
    ecrivain = {}
    ecrivain['dbpedia-en'] = e[0]
    ecrivain['naissance'] = e[2] 

    if nom not in ecrivains_temp:
        ecrivains_temp[nom] = [ecrivain]
    else:
        if not trouver_ecrivain(ecrivain, ecrivains_temp[nom]):
            ecrivains_temp[nom].append(ecrivain)
            
for nom in ecrivains_temp:    
    if nom not in ecrivains:
        ecrivains[nom] = ecrivains_temp[nom]
    else:
       # Aucun moyen de trouver le bon écrviain s'il y en a plus d'un. On va troujours
        # prendre le premier et supposer que c'est celui-là qu'on a trouvé dans DBpedia anglais
        ecrivains[nom][0]['dbpedia-en'] = ecrivains_temp[nom][0]['dbpedia-en'] 
        if ecrivains[nom][0]['naissance'] == 'X':
                    ecrivains[nom][0]['naissance'] = ecrivains_temp[nom][0]['naissance'] 


print(len(ecrivains),'écrivains au total')
print(len([e for es in ecrivains.values() for e in es if 'dbpedia-en' in e 
                                   and 'dbpedia-fr' not in e]), 'ecrivains sont dans DBpedia anglais seulement')
print(len([e for es in ecrivains.values() for e in es if 'dbpedia-en' not in e 
                                   and 'dbpedia-fr' in e]), 'ecrivains sont dans DBpedia français seulement')

print(len([e for es in ecrivains.values() for e in es if 'dbpedia-fr' in e
                                   and 'dbpedia-en' in e]), 'ecrivains sont dans DBpedia français et anglais')

print(len([e for es in ecrivains.values() for e in es if e['naissance'] != 'X']), 'ecrivains ont une date de naissance spécifiée')
print(len([e for es in ecrivains.values() for e in es if e['naissance'] != 'X' 
                                   and 'dbpedia-en' in e]), 'ecrivains ont une date de naissance spécifiée dans DBpedia anglais')
print(len([e for es in ecrivains.values() for e in es if e['naissance'] != 'X' 
                                   and 'dbpedia-fr' in e]), 'ecrivains de DBpedia français ont une date de naissance spécifiée après voir extrait les données de DBpedia anglais')



         
# print(list(ecrivains.items())[:10])

file_writers = open("./ecrivains_dbpedia_en.txt","w")
for e in ecrivains_dbpedia_en:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()
    
# for e in ecrivains_dbpedia_en:
#     print(e)

# for e in list(ecrivains.items()):
#      print(e[0])

    
    

784 écrivains du Québec trouvés dans DBpedia anglais
1738 écrivains au total
485 ecrivains sont dans DBpedia anglais seulement
962 ecrivains sont dans DBpedia français seulement
297 ecrivains sont dans DBpedia français et anglais
843 ecrivains ont une date de naissance spécifiée
669 ecrivains ont une date de naissance spécifiée dans DBpedia anglais
459 ecrivains de DBpedia français ont une date de naissance spécifiée après voir extrait les données de DBpedia anglais


Deux écrivains sont dédoublés parce qu'ils ont deux dates de naissances différentes (erreurs dans DBpedia anglais)
* Victor Barbeau
* Angeline Hango

Ces doublons sont retirés lorsqu'on les ajoute à la liste d'écrvains (on n'ajoute pas un écrivain si son URI s'y trouve déjà)


À noter qu'il y a deux écrivains qui ont deux homonymes dans DBpedia français et qui se trouvent aussi dans DBpedia anglais:
* Pierre Morency
* Louis Émond

Le hasard a voulu que le premier de la liste corresponde à celui de DBpedia anglais pour Pierre Morency. 
Malheureusment, pour Louis Émond, le premier n'est pas le bon. 

### Problèmes identifiés:
    
* Alexander Henry dédoublé avec Alexander Henry the elder 
* Jean Basile Bezroudnoff ecrit sans son dernier nom  dans Wikipedia anglais, contrairement à Wikipedia francais)
* Qu'est-ce que La Butte à Mathieu vient faire là?
* On retrouve les URI suivantes:
    * List_of_Quebec_writers
    * List_of_Quebec_writers
    * Liste_de_poètes_québécois
* Dans certains cas, le nom utilisé pour l'URI de DBpedia anglais n'est pas le même que puour le français. :Par exemple: Gaétan_Brulotte (DBpedia anglais) vs Gaëtan_Brulotte (DBpedia anglais). Dans ce cas-ci il y a des sameAs qui sont définis - Pierre DesRuisseaux est Pierre Desruisseaux dans DBpedia français
* Jean Charbonneau: il y a un poète du 19e siècle, celui que nous avons trouvé dans Wikidata, et un auteur contemporain, que nous avons trouvé dans DBpedia, ce qui fait une entrée incohérente

### Recherche des auteur hors Québec dans DBpedia

Plus tard, quand on tentera de retrouver les auteurs dans Wikidata, il sera difficile de déterminer lesquels
sont des auteurs francophones hors Québec. Pour les filtrer, il nous faut une liste de ces auteurs, qu'on 
obtiendra à partir de DBpdedia.



In [9]:
# Dans DBpedia anglais
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   SELECT DISTINCT ?s ?label 
   WHERE {
     ?s dcterms:subject ?cat .
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Alberta> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_British_Columbia> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Manitoba> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_New_Brunswick> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Newfoundland_and_Labrador> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_the_Northwest_Territories> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Nova_Scotia> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Nunavut> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Ontario> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Prince_Edward_Island> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Saskatchewan> }
     UNION
     { ?cat skos:broader+ <http://dbpedia.org/resource/Category:Writers_from_Yukon> }
     
     ?s rdfs:label ?label . 
     FILTER (lang(?label) = "fr")
    } 
""")
sparql.setReturnFormat(JSON)
results_hors_quebec = sparql.query().convert()

In [10]:
ecrivains_dbpedia_en_hors_quebec = [(b['s']['value'],traiter_nom(b['label']['value']))
                            for b in results_hors_quebec['results']['bindings']]
print(len(ecrivains_dbpedia_en_hors_quebec), 'écrivains hors Québec dans DBpedia anglais')
   
 

1122 écrivains hors Québec dans DBpedia anglais


In [11]:
# Dans DBpedia français
sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setQuery("""
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   SELECT DISTINCT ?s ?label 
   WHERE {
     ?s dcterms:subject ?cat .
     { ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_canadien_francophone_hors_Québec> }
     UNION
     { ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_franco-ontarien> }
     UNION
     { ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_néo-brunswickois> }
     UNION
     { ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_néo-écossais> }
     
     ?s rdfs:label ?label . 
     FILTER (lang(?label) = "fr")
    } 
""")
sparql.setReturnFormat(JSON)
results_fr_hors_quebec = sparql.query().convert()

In [12]:
ecrivains_dbpedia_fr_hors_quebec = [(b['s']['value'],traiter_nom(b['label']['value']))
                            for b in results_fr_hors_quebec['results']['bindings']]
print(len(ecrivains_dbpedia_fr_hors_quebec), 'écrivains hors Québec dans DBpedia français')
     
 

113 écrivains hors Québec dans DBpedia français


## Wikidata

### Recherche des auteurs dans Wikidata

Les classes potentiellement pertinentes dans Wikidata sont les suivantes:

* Q36180 Écrivain
   * wd:Q28389	scénariste
   * wd:Q49757	poète
   * wd:Q214917	dramaturge
   * wd:Q381353	femme de lettres
   * wd:Q822146	parolier
   * wd:Q6625963	romancier
   * wd:Q4853732	auteur de littérature pour la jeunesse
   * wd:Q5434338	écrivain de fantasy
   * wd:Q1626130	homme de lettres
   * wd:Q3589290	épistolier
   * wd:Q3064032	fabuliste
   * wd:Q10429346	bibliographe
   * wd:Q10297252	auteur de roman policier
   * wd:Q11313148	écrivain sportif
   * wd:Q21140478	écrivain régionaliste
   * wd:Q26203955	auteur de contes
   * wd:Q15949613	nouvelliste
   * wd:Q16254673	pamphlétaire
   * wd:Q15980158	essayiste
   * wd:Q18844224	écrivain de science-fiction

On va donc tenter de retrouver les personnes qui sont des instances de ces classes, nées au Canada et dont la langue est le français.


In [13]:
# requete pour obtenir toutes les personnes du Canada (Q16) ayant pour langue le français (Q150) ou le français canadien (Q1450506)
# et qui sont des instances de writer (Q36180) ou d'une de ses sous-classes parmi les suivantes: poète (Q49757), 
# dramaturge (Q214917), femme de lettre (femme de lettres), romancier (Q6625963), auteur de littérature pour la 
# jeunesse (Q4853732), écrivain de fantasy (Q5434338), homme de lettres (Q1626130), auteur de roman policier
# (Q10297252), écrivain de science-fiction (wd:Q18844224), essayiste (wd:Q15980158), auteur de contes (Q26203955)
# Si aucune des catégories précédente n'est trouvée, on accepte ceux qui on un ID VIAF


q_all = """
SELECT  DISTINCT ?s ?l
WHERE 
{
  { ?s wdt:P27 wd:Q16 } UNION { ?s wdt:P27 wd:Q16017119 } 
   
  { ?s wdt:P1412|wdt:P6886 wd:Q150} UNION {?s wdt:P1412|wdt:P6886 wd:Q1450506 }
  
  {?s rdfs:label ?l }

  FILTER (lang(?l) = "fr")


  { ?s wdt:P106  wd:Q36180 } UNION    
  { ?s wdt:P106  wd:Q49757 } UNION
  { ?s wdt:P106  wd:Q214917 }  UNION
  { ?s wdt:P106  wd:Q381353 } UNION
  { ?s wdt:P106  wd:Q6625963 } UNION
  { ?s wdt:P106  wd:Q4853732 } UNION
  { ?s wdt:P106  wd:Q5434338 } UNION
  { ?s wdt:P106  wd:Q1626130 } UNION
  { ?s wdt:P106  wd:Q10297252 } UNION 
  { ?s wdt:P106  wd:Q18844224 } UNION 
  { ?s wdt:P106  wd:Q15980158 } UNION 
  { ?s wdt:P106  wd:Q26203955 } UNION
  { ?s wdt:P214 [] }
  
}
"""


rep_wikidata = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q_all)

if rep_wikidata.status_code != 200:
    print('ERREUR DANS LA REQUÊTE SPARQL')



In [14]:
ecrivains_wikidata = [(b['s']['value'],b['l']['value']) for b in rep_wikidata.json()['results']['bindings']]
print(len(ecrivains_wikidata), 'entités trouvées qui sont instances de writer ou d\'une de ses sous-classes')
print(len([e for e in ecrivains_wikidata if e[1] not in ecrivains]), 'écrivains qui ne sont pas dans la liste extraite de DBpedia')


# On ajoute à la liste des écrivains 
for e in ecrivains_wikidata:
    if e[1] in ecrivains:
        ecrivains[e[1]][0]['wikidata'] = e[0]

# for e in ecrivains:
#     if 'wikidata' not in ecrivains[e][0]:
#         print(ecrivains[e])

file_writers = open("./ecrivains_wikidata.txt","w")
for e in ecrivains_wikidata:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()

#print(ecrivains_wikidata[:10])

        
#print(list(ecrivains.items())[:10])



6603 entités trouvées qui sont instances de writer ou d'une de ses sous-classes
5351 écrivains qui ne sont pas dans la liste extraite de DBpedia


Analyse des écrivains de Wikidata qui ne se retrouvent pas dans la liste extratite de DBpdedia:

* De fait, on les retrouve presque tous dans DBpedia (français, anglais ou les deux)
* Près du quart sont effectivement des écrivains québécois (les autres sont en général des auteurs hors Québec, des universitaires, ou des gens qui ne sont tout simplement pas écrivains)
* Parmi les écrivains québécois, leur absence de la liste peut être expliquée par les raisons suivantes:
    * Erreur dans l'écriture du nom, faisans en sorte que le nom ne correspond pas à l'URI dans DBpedia (Marc-André au lieu de Marc André, par exemple)
    * La personne est présente dans Wikipedia mais pas dans DBpedia
    * Dans la plupart des cas, les catégories associées à la personne n'ont pas permis de l'identifier clairement comme écrivain
* Parmi les écrivains québécois, vraiment très peu sont membres de l'UNEQ (ce qui signifie que cela ne nous aiderait pas à les détecter)

Il serait risqué d'utiliser cette liste extraire de Wikidata, puisqu'on aura environ 4000 instances qui ne sont pas des écrivains québécois

On va donc plutôt faire le contraire: on va tenter de prendre les écrivains que nous avons extrait de DBpedia et les retrouver dans Wikidata

In [20]:
from time import sleep
from ecrivains_pas_dans_wikidata import pas_dans_wikidata
from ecrivains_dans_wikidata import dans_wikidata

q_ecrivain = """
SELECT  DISTINCT ?s 
WHERE 
{{

  {{ ?s rdfs:label "{0}"@fr }} UNION {{ ?s rdfs:label "{0}"@en }} UNION {{ ?s skos:altLabel "{0}"@fr }}
  
  {{ ?s wdt:P106  wd:Q36180 }} UNION    
  {{ ?s wdt:P106  wd:Q49757 }} UNION
  {{ ?s wdt:P106  wd:Q214917 }} UNION
  {{ ?s wdt:P106  wd:Q381353 }} UNION
  {{ ?s wdt:P106  wd:Q6625963 }} UNION
  {{ ?s wdt:P106  wd:Q4853732 }} UNION
  {{ ?s wdt:P106  wd:Q5434338 }} UNION
  {{ ?s wdt:P106  wd:Q1626130 }} UNION
  {{ ?s wdt:P106  wd:Q10297252 }} UNION 
  {{ ?s wdt:P106  wd:Q18844224 }} UNION 
  {{ ?s wdt:P106  wd:Q15980158 }} UNION 
  {{ ?s wdt:P106  wd:Q26203955 }} UNION
  {{ ?s wdt:P106  wd:Q11774202 }} UNION
  {{ ?s wdt:P106  wd:Q1930187 }} UNION
  {{ ?s wdt:P106  wd:Q487596 }} UNION
  {{ ?s wdt:P106  wd:Q201788 }}

}}
"""

for e in ecrivains:
#     if e in pas_dans_wikidata:
#         continue 
#     if 'wikidata' not in ecrivains[e][0]:
#         if e in dans_wikidata:
#              ecrivains[e][0]['wikidata'] = dans_wikidata[e]
#         else:
            sleep(1)
            print("---" + e + "----")
            rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q_ecrivain.format(e))
            if rep_wikidata_ecrivain.status_code == 200:
                if len(rep_wikidata_ecrivain.json()['results']['bindings']) == 1:
                    ecrivains[e][0]['wikidata'] = rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value']
                    print(rep_wikidata_ecrivain.json()['results']['bindings'][0]['s'])
            else:
                print('ERREUR:',e)


---Véronique Cyr----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3564298'}
---Laurent McAllister----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3219386'}
---Thérèse Sauvageau----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3527371'}
---Laurent Poliquin----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3219457'}
---Fernande Saint-Martin----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3069412'}
---Ollivier Dyens----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3351513'}
---Olivier Choinière----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3350914'}
---Jacques Languirand----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3159260'}
---Edem Awumey----
{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3047384'}


KeyboardInterrupt: 

In [13]:
print(len([e for e in ecrivains]), 'noms d\'écrivains au total')
print(len([e for e in ecrivains if 'wikidata' in ecrivains[e][0]]), 'ont été trouvé dans Wikidata')


1738 noms d'écrivains au total
1570 ont été trouvé dans Wikidata


## L'ÎLE - Infocentre littéraire des écrivains québécois

Centre de documentation virtuel qui a pour mission de promouvoir la littérature québécoise. Il s’adresse aux étudiants, aux chercheurs, aux journalistes et à tous ceux qui s’intéressent aux écrivains québécois et à la littérature québécoise.

Regroupe des informations sur des écrivains québécois ou canadiens d’expression française, auteurs d’au moins deux livres d’au moins 48 pages, publiés par une maison d’édition reconnue, dont les livres appartiennent à l’un ou l’autre des genres suivants :
* le roman
* le récit
* la nouvelle
* le conte
* la poésie
* le théâtre
* l’essai ou l’étude
* la bande dessinée
* la biographie
* la littérature jeunesse


Le site fournit aussi une très grande liste à des sites externes (librairies, maisons d'édition, sites personnels des auteurs, associations, etc.)

###  Liste des écrivains extraite le 11 décembre 2019:

La liste des auteurs a été extraite manuellement à partir du site, et placée dans une liste Python
se trouvant dans le fichiers auteurs_ile.py

In [17]:
from auteurs_ile import auteurs
print(len(auteurs), 'auteurs recensés')

1769 auteurs recensés


In [18]:
def transformer_nom(chaine):
    (nom, prenom) = chaine.split(", ")
    return prenom.strip() + " " + nom.strip()


# Pour normaliser, on transforme le nom de la manière suivante:
#   Nom, Prénom --> Prénom Nom
# Et on met tout en minuscule
auteurs_ile_transf = [transformer_nom(a) for a in auteurs]
    

# print(auteurs_ile_transf[2])
# print(auteurs_ile_transf[4])
# print(auteurs_ile_transf[10])
# print(auteurs_ile_transf[43])
# print(auteurs_ile_transf[56])

print(len(auteurs_ile_transf), 'auteurs dans ILE')
print(len([e for e in auteurs_ile_transf if e in ecrivains]), 'auteurs dans ILE trouvés dans DBpedia')

# On ajoute à la liste des écrivains 
for e in auteurs_ile_transf:
    if e in ecrivains:
        ecrivains[e][0]['ILE'] = True
    else:
        ecrivains[e] = [{'ILE':True}]
 

print(len(ecrivains), 'noms d\'auteurs au total après avoir ajouté ceux de ILE')


# for e in ecrivains:
#     if 'ILE' in ecrivains[e][0]:
#         print(ecrivains[e])
        
        

1769 auteurs dans ILE
657 auteurs dans ILE trouvés dans DBpedia
2849 noms d'auteurs au total après avoir ajouté ceux de ILE



Remarque: il y a deux Louis Émond dans ILE:
* Émond (1957), Louis
* Émond (1969),  Louis
Comme ils contiennent une date pour les distinguer, il n'ont pas été retrouvés comme tel dans la liste d'écrivains. Ils ont donc été ajoutés comme nouvelles entités. 

Autres problèmes notés:
* Jean Basile Bezroudnoff écrit sans son dernier nom (comme dans Wikipedia anglais, mais contrairement à Wikipedia francais)
* Jean-Pierre Petits écrit avec un "s" alors qu'il est sans "s" dans UNEQ. Lequel est le bon?
* Ph. (Philippe) Aubert de Gaspé - Correspond à Philippe Aubert de Gaspé, fils

Les erreurs suivantes ont dû être corrigées dans la liste de l'ILE:

* Alfred Desrochers écrit sans R majuscules (devrait être DesRochers)
* Claude R Blouin (manque le point après R)
* Louis-Adolphe Pâquet écrit accent circonflexe seulement dans ILE
* Fredrick D'Anterny devrait être écrit avec d minuscule
* H. (Honoré) Beaugrand vs "Honoré Beaugrand" dans Wikipedia
* H.-R. (Henri-Raymond) Casgrain vs "Henri-Raymond Casgrain" dans Wikipedia
* Huguette Brun (Gauthier) - Sans "Gauthier" dans UNEQ
* Jacques Benoit écrit sans accent circonflexe
* Joanne Morncy (semble être une erreur, puisqu'il y a une Joanne Morency avec la même fiche)
* Markendy E.J. Simon - Il n'y a pas les initiales "E.J" dans ILE
* Mona Latif Ghattas - Trait d'union manquant dans Latif-Ghattas
* Narcisse Henri Édouard Faucher de Saint-Maurice - Il manque les tirets dans le prénom
* Pamphile LeMay - devrait être écrit Le May
* Réal Benoit - accent circonflexe manquant sur le i




## UNEQ


### Liste des écrivains extraite le 12 décembre 2019

In [19]:
from membres_uneq import membres_uneq
print(len(membres_uneq), 'écrivains membres de l\'UNEQ')

def minusculiser(chaine):
    ch = chaine.split()
    resultat = ''
    for c in ch:
        if '-' in c:
            ch2 = c.split(sep = '-')
            ch2 = [c[:1] + c[1:].lower() for c in ch2]
            resultat += ' ' + '-'.join(ch2)
        elif c.find("'") != -1:
            ch2 = c.split(sep = '\'')
            ch2 = [c[:1] + c[1:].lower() for c in ch2]
            resultat += ' ' + '\''.join(ch2)
        else:
            resultat += ' ' + c[:1] + c[1:].lower()
    return resultat.strip()


def transformer_nom(chaine):
    (nom, prenom) = chaine.split(",")
    if prenom != '':
        prenom = prenom.strip() + ' '
    return prenom + nom.strip()

membres_uneq_transf = [transformer_nom(a) for a in membres_uneq]



# On ajoute à la liste des écrivains 
for e in membres_uneq_transf:
    if e in ecrivains:
        ecrivains[e][0]['UNEQ'] = True
    else:
        ecrivains[e] = [{'UNEQ':True}]
    
# for e in ecrivains:
#     if 'UNEQ' in ecrivains[e][0] and ecrivains[e][0]['UNEQ'] == True:
#         print(e)

# for e in ecrivains:
#     if 'ILE' in ecrivains[e][0]:
#         print(ecrivains[e])


print(len(ecrivains), 'noms d\'auteurs au total après avoir ajouté ceux de UNEQ')



1613 écrivains membres de l'UNEQ
3589 noms d'auteurs au total après avoir ajouté ceux de UNEQ


In [20]:
def extraire_id(entite):
    if entite:
        return entite.split('/')[-1]
    else:
        return 'X'
      

file_writers = open("./ecrivains_liste_complete.txt","w") 
file_writers.write('NOM|ILE|UNEQ|WIKIDATA|DBPEDIA-EN|DBPEDIA-FR\n')
for nom in sorted(ecrivains):
    liste_e = ecrivains[nom]
    for e in liste_e:
        file_writers.write('{}|{}|{}|{}|{}|{}\n'.format(nom,
                               'ILE' if 'ILE' in e else 'X', 
                               'UNEQ' if 'UNEQ' in e else 'X',               
                               extraire_id(e.get('wikidata')),
                               extraire_id(e.get('dbpedia-en')),
                               extraire_id(e.get('dbpedia-fr'))))
file_writers.close()



Problème trouvés dans la liste des membres UNEQ:

* Rose Hélène Tremblay - Il faut un tiret dans le prénom (corrigé dans la liste finale)
* Jovette R. Bernier - Pas de R. dans Wikipedia


Dans la création de la liste minusculisée des écrivains de l'UNEQ, il a fallu faire attention aux cas suivants:

* Jean-Simon Desrochers devrait être écrit "DesRochers"
* Kim Thuy - Il faut un accent sur le "u"
* Markendy E.j. Simon (le "j" ne devrait pas être minuscule - bogue)
* Monique Larue - Devrait être LaRue
* Rose Hélène Tremblay - Il faut un tiret dans le prénom


In [21]:
print('Quelques statistiques:')

print('---------------------------------------------')


print(len(ecrivains),'noms d\'écrivain au total')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' in ecrivains[e][0] 
                                    and 'ILE'  in ecrivains[e][0] 
                                    and 'wikidata'  in ecrivains[e][0]
                                    and 'dbpedia-fr'  in ecrivains[e][0] 
                                    and 'dbpedia-en'  in ecrivains[e][0] 

          ]),'écrivains se retrouvent dans toutes les sources')


print('---------------------------------------------')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' in ecrivains[e][0]]),
      'écrivains dans UNEQ')
print(len([ecrivains[e][0] for e in ecrivains if 'ILE' in ecrivains[e][0]]),
      'écrivains dans ILE')

print(len([ecrivains[e][0] for e in ecrivains if 'ILE' in ecrivains[e][0] and ('dbpedia-fr' in ecrivains[e][0] 
                                    or 'dbpedia-en' in ecrivains[e][0])]),
      'écrivains dans ILE et DBpedia')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' in ecrivains[e][0] and ('dbpedia-fr' in ecrivains[e][0] 
                                    or 'dbpedia-en' in ecrivains[e][0])]),
      'écrivains dans UNEQ et DBpedia')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ'  in ecrivains[e][0] 
                                    and 'ILE' in ecrivains[e][0]  
          ]),'écrivains se retrouvent dans ILE et UNEQ')



print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' in ecrivains[e][0] 
                                    and 'ILE' not in ecrivains[e][0] 
                                    and 'wikidata' not in ecrivains[e][0]
                                    and 'dbpedia-fr' not in ecrivains[e][0] 
                                    and 'dbpedia-en' not in ecrivains[e][0] 

          ]),'écrivains se retrouvent seulement dans UNEQ')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' not in ecrivains[e][0] 
                                    and 'ILE' in ecrivains[e][0] 
                                    and 'wikidata' not in ecrivains[e][0]
                                    and 'dbpedia-fr' not in ecrivains[e][0] 
                                    and 'dbpedia-en' not in ecrivains[e][0] 

          ]),'écrivains se retrouvent seulement dans ILE')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ'  in ecrivains[e][0] 
                                    and 'ILE' in ecrivains[e][0] 
                                    and 'wikidata' not in ecrivains[e][0]
                                    and 'dbpedia-fr' not in ecrivains[e][0] 
                                    and 'dbpedia-en' not in ecrivains[e][0] 

          ]),'écrivains se retrouvent seulement dans ILE et UNEQ')



print('---------------------------------------------')
print(len([ecrivains[e][0] for e in ecrivains if  ('dbpedia-fr' in ecrivains[e][0] 
                                    or 'dbpedia-en' in ecrivains[e][0])

          ]),'écrivains se retrouvent dans DBpedia (anglais ou français)')

print(len([ecrivains[e][0] for e in ecrivains if  'dbpedia-en' in ecrivains[e][0]

          ]),'écrivains se retrouvent dans DBpedia anglais')


print(len([ecrivains[e][0] for e in ecrivains if  'dbpedia-fr' in ecrivains[e][0]

          ]),'écrivains se retrouvent dans DBpedia français')


print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' not in ecrivains[e][0] 
                                    and 'ILE' not in ecrivains[e][0] 
                                    and 'wikidata' not in ecrivains[e][0]
                                    and ('dbpedia-fr' in ecrivains[e][0] 
                                    or 'dbpedia-en' in ecrivains[e][0])

          ]),'écrivains se retrouvent seulement dans DBpedia (anglais ou français)')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' not in ecrivains[e][0] 
                                    and 'ILE' not in ecrivains[e][0] 
                                    and 'wikidata' not in ecrivains[e][0]
                                    and 'dbpedia-en' in ecrivains[e][0]

          ]),'écrivains se retrouvent seulement dans DBpedia anglais')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' not in ecrivains[e][0] 
                                    and 'ILE' not in ecrivains[e][0] 
                                    and 'wikidata' not in ecrivains[e][0]
                                    and 'dbpedia-fr' in ecrivains[e][0]

          ]),'écrivains se retrouvent seulement dans DBpedia français')

print('---------------------------------------------')

print(len([ecrivains[e][0] for e in ecrivains if 'wikidata' in ecrivains[e][0]
          ]),'écrivains se retrouvent dans Wikidata')

print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' not in ecrivains[e][0] 
                                    and 'ILE' not in ecrivains[e][0] 
                                    and 'wikidata' in ecrivains[e][0]
                                    and 'dbpedia-fr' not in ecrivains[e][0] 
                                    and 'dbpedia-en' not in ecrivains[e][0] 

          ]),'écrivains se retrouvent seulement dans Wikidata')

print(len([ecrivains[e][0] for e in ecrivains if 'wikidata'  in ecrivains[e][0]
                                    and ('dbpedia-fr' in ecrivains[e][0] 
                                    or 'dbpedia-en' in ecrivains[e][0])

          ]),'écrivains se retrouvent dans Wikidata et DBpedia')


print(len([ecrivains[e][0] for e in ecrivains if 'UNEQ' not in ecrivains[e][0] 
                                    and 'ILE' not in ecrivains[e][0] 
                                    and 'wikidata'  in ecrivains[e][0]
                                    and ('dbpedia-fr' in ecrivains[e][0] 
                                    or 'dbpedia-en' in ecrivains[e][0])

          ]),'écrivains se retrouvent seulement dans Wikidata et DBpedia')


print('---------------------------------------------')






Quelques statistiques:
---------------------------------------------
3589 noms d'écrivain au total
76 écrivains se retrouvent dans toutes les sources
---------------------------------------------
1610 écrivains dans UNEQ
1765 écrivains dans ILE
654 écrivains dans ILE et DBpedia
320 écrivains dans UNEQ et DBpedia
823 écrivains se retrouvent dans ILE et UNEQ
740 écrivains se retrouvent seulement dans UNEQ
561 écrivains se retrouvent seulement dans ILE
550 écrivains se retrouvent seulement dans ILE et UNEQ
---------------------------------------------
1738 écrivains se retrouvent dans DBpedia (anglais ou français)
782 écrivains se retrouvent dans DBpedia anglais
1253 écrivains se retrouvent dans DBpedia français
161 écrivains se retrouvent seulement dans DBpedia (anglais ou français)
144 écrivains se retrouvent seulement dans DBpedia anglais
17 écrivains se retrouvent seulement dans DBpedia français
---------------------------------------------
1570 écrivains se retrouvent dans Wikidata
0

## Auteurs contemporains

Ce site fournit un ensemble de documents critiques sur certains écrivains (environ 80), pas tous québécois.
Pour chaque auteur, une fiche fournit:

* Une liste des prix reçus
* Des liens vers des sites externes
* Des références à des documents critiques pour certaines oeuvres spécifiques et pour l'ensemble le l'oeuvre. Pas d'hyperliens sur les critiques elles-mêmes. Il semble y avoir les hyperliens pour les articles se retrouvant sur Érudit. Mais visiblement, il en manque beaucoup. Par exemple on ne retrouve pas l'article que Marc Chabot a écrit sur Gilles Archambault dans la revue Nuit blanche.
* Une bibliographie complète (titres des oeuvres, sans hyperliens)

## Les poètes de l'Amérique française

Les items intéressants qu'on retrouve sur ce site:

* Programmation des prochains événements
* Une banque de séquences audio de textes qui ont été récités


# A faire

* Traiter les pseudonymes (parfois, comme pour "Laurent McAllister", plusieurs écrivains se cachent sous un pseudonyme

* Trouver une vraie solution pour les homonymes

In [21]:
dans_wikidata = {}

# for e in ecrivains:
#     if 'wikidata' in ecrivains[e][0]:
#         print("\"{}\":\"{}\",".format(e,ecrivains[e][0]['wikidata']))